In [ ]:
import argparse
import random
import time
import datetime
import socket
import math
import re
import uuid
from ipaddress import IPv4Address


class Log:
    def __init__(self):
        self.timestamp = ""
        self.id = ""
        self.src = ""
        self.src_port = 0
        self.dest = ""
        self.dest_port = 0
        self.bytes = 0
        self.proto = ""
        self.duration = ""


def generate_uid():
    return uuid.uuid4().hex


def random_network_delay():
    return random.randint(100, 500)


def generate_biased_triangular_random(min, max):
    mode = (min + max) / 2.0
    mode_bias = 0.8
    r = random.random()

    if r < mode_bias:
        return min + math.sqrt(r * (max - min) * (mode - min))
    else:
        return max - math.sqrt((1 - r) * (max - min) * (max - mode))


def generate_internal_ip():
    # 10.0.0.0 to 10.255.255.255
    return IPv4Address(random.randint(0x0A000000, 0x0AFFFFFF))


def generate_public_ip():
    public_ranges = [
        IPv4Address("8.120.2.10"),
        IPv4Address("17.64.4.20"),
        # Add other ranges as needed...
    ]
    return str(random.choice(public_ranges))


def parse_jitter(jitter):
    pattern = r'(\d+)s-(\d+)%'
    match = re.search(pattern, jitter)

    if not match:
        raise ValueError("Invalid jitter format. Must be like '60s-10%'.")

    sleep_seconds, sleep_percentage = match.groups()
    return float(sleep_seconds), float(sleep_percentage) / 100


def write_log(log):
    print(f"{log.timestamp} {log.id} {log.src} {log.src_port} {log.dest} {log.dest_port} {log.bytes} {log.proto}")


def beacon(source_ip, destination_ip, destination_port, protocol, bytes, start_time, duration, jitter):
    first_beacon_time = start_time
    while True:
        log = Log()

        log.id = generate_uid()
        log.src = source_ip
        log.dest = destination_ip
        log.dest_port = destination_port
        log.proto = protocol
        log.bytes = bytes
        log.src_port = random.randint(1024, 65535)

        seconds, percentage = parse_jitter(jitter)
        sleep_percentage = percentage * seconds
        sleep_seconds = random.uniform(
            seconds - sleep_percentage, seconds + sleep_percentage)
        next_beacon = start_time + datetime.timedelta(seconds=sleep_seconds)
        next_beacon_with_net_delay = next_beacon + \
            datetime.timedelta(milliseconds=random_network_delay())

        log.timestamp = next_beacon_with_net_delay.strftime(
            "%Y-%m-%dT%H:%M:%SZ")
        write_log(log)

        if next_beacon > first_beacon_time + duration:
            break

        start_time = next_beacon


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "-source", default=generate_internal_ip(), help="Source IP address")
    parser.add_argument(
        "-destination", default=generate_public_ip(), help="Destination IP address")
    parser.add_argument("-port", default=443, type=int,
                        help="Destination port")
    parser.add_argument("-protocol", default="tls", help="Protocol")
    parser.add_argument("-bytes", default=221, type=int,
                        help="number of bytes")
    parser.add_argument("-starttime", default=datetime.datetime.now().isoformat(),
                        help="Start Time, Example: 2023-10-20T15:10:10")
    parser.add_argument("-duration", default=7*24*60*60,
                        type=int, help="duration in seconds (Default 1 week)")
    parser.add_argument("-jitter", required=True,
                        help="Jitter: sleep-percentage -> (e.g 60s-10%)")
    args = parser.parse_args()

    start_time = datetime.datetime.fromisoformat(args.starttime)
    duration = datetime.timedelta(seconds=args.duration)

    beacon(args.source, args.destination, args.port, args.protocol,
           args.bytes, start_time, duration, args.jitter)


if __name__ == "__main__":
    main()